In [1]:
%env ALL_PROXY=http://127.0.0.1:7890
%env HTTP_PROXY=http://127.0.0.1:7890
%env HTTPS_PROXY=http://127.0.0.1:7890

env: ALL_PROXY=http://127.0.0.1:7890
env: HTTP_PROXY=http://127.0.0.1:7890
env: HTTPS_PROXY=http://127.0.0.1:7890


In [2]:
!ls ~/.cache/huggingface/hub

datasets--allenai--c4
datasets--bentrevett--multi30k
datasets--ccdv--arxiv-summarization
datasets--common-pile--arxiv_papers
datasets--datablations--c4-subsets
datasets--erikanesse--great_books
datasets--gsm8k
datasets--hendrycks_test
datasets--ILSVRC--imagenet-1k
datasets--izumi-lab--open-text-books
datasets--izumi-lab--wikipedia-en-20230720
datasets--lansinuote--peoples-daily-ner
datasets--mnist
datasets--openai--gsm8k
datasets--peoples_daily_ner
datasets--peoples-daily-ner--peoples_daily_ner
datasets--teven--c4_15M
models--bert-base-german-dbmdz-cased
models--bert-base-uncased
models--deepseek-ai--Deepseek-R1-Distill-Llama-8B
models--deepseek-ai--Deepseek-R1-Distill-Qwen-7B
models--distilbert--distilbert-base-uncased-finetuned-sst-2-english
models--facebook--opt-125m
models--facebook--opt-350m
models--google--owlvit-base-patch32
models--hfl--chinese-macbert-base
models--Johnson8187--Chinese-Emotion
models--meta-llama--Llama-3.1-8B
models--meta-llama--Llama-3.1-8b-Instruct
models--me

# NER (Named Entity Recognition) based-on transformers

## Step01 import related packages

In [3]:
import evaluate
from datasets import load_dataset
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer

/home/ubuntu/MyFiles/GitHub/demos/transformers/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step 02 Load the datset

In [4]:
ner_ds_dict = load_dataset("lansinuote/peoples-daily-ner")
ner_ds_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 4637
    })
})

In [5]:
ner_ds_dict['train'][0]

{'id': '0',
 'tokens': ['海',
  '钓',
  '比',
  '赛',
  '地',
  '点',
  '在',
  '厦',
  '门',
  '与',
  '金',
  '门',
  '之',
  '间',
  '的',
  '海',
  '域',
  '。'],
 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0]}

In [6]:
ner_ds_dict['train'].features

{'id': Value('string'),
 'tokens': List(Value('string')),
 'ner_tags': List(ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']))}

In [7]:
label_list = ner_ds_dict['train'].features['ner_tags'].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

## Pre-process dataset

In [8]:
tokenizer = AutoTokenizer.from_pretrained('hfl/chinese-macbert-base')

In [9]:
tokenizer(ner_ds_dict['train'][0]['tokens'])

{'input_ids': [[101, 3862, 102], [101, 7157, 102], [101, 3683, 102], [101, 6612, 102], [101, 1765, 102], [101, 4157, 102], [101, 1762, 102], [101, 1336, 102], [101, 7305, 102], [101, 680, 102], [101, 7032, 102], [101, 7305, 102], [101, 722, 102], [101, 7313, 102], [101, 4638, 102], [101, 3862, 102], [101, 1818, 102], [101, 511, 102]], 'token_type_ids': [[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [10]:
tokenizer(ner_ds_dict['train'][0]['tokens'], is_split_into_words=True) # <- Must order is_split_into_words=True, when using the datasets which have already been tokenized

{'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
res = tokenizer('interesting word')
res

{'input_ids': [101, 10673, 12865, 12921, 8181, 8681, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [12]:
print(tokenizer.convert_ids_to_tokens(res['input_ids']))

['[CLS]', 'int', '##ere', '##stin', '##g', 'word', '[SEP]']


In [13]:
res.word_ids()

[None, 0, 0, 0, 0, 1, None]

In [14]:
def process_function(examples):
    """Do the label mapping with word_ids"""
    tokenized_examples = tokenizer(examples['tokens'], max_length=128, truncation=True, is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_examples.word_ids(batch_index=idx)
        label_ids = []
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100) # <-- -100 is a special value which will be ignored in PyTorch's CrossEntropyLoss.
            else:
                label_ids.append(label[word_id])
        labels.append(label_ids)
    tokenized_examples['labels'] = labels
    return tokenized_examples

In [15]:
tokenized_ds_dict = ner_ds_dict.map(process_function, batched=True)
tokenized_ds_dict

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 20865
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 2319
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 4637
    })
})

In [16]:
print(tokenized_ds_dict['train'][0])

{'id': '0', 'tokens': ['海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间', '的', '海', '域', '。'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0], 'input_ids': [101, 3862, 7157, 3683, 6612, 1765, 4157, 1762, 1336, 7305, 680, 7032, 7305, 722, 7313, 4638, 3862, 1818, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 5, 6, 0, 5, 6, 0, 0, 0, 0, 0, 0, -100]}


## Step 04 Create the model

In [17]:
model = AutoModelForTokenClassification.from_pretrained('hfl/chinese-macbert-base', num_labels=len(label_list)) # Must order num_labels for any non-binary classification tasks

Some weights of BertForTokenClassification were not initialized from the model checkpoint at hfl/chinese-macbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model.config.num_labels

7

## Step 05 Create the evaluation function

In [19]:
seqeval = evaluate.load('seqeval')
seqeval

EvaluationModule(name: "seqeval", module_type: "metric", features: {'predictions': List(Value('string')), 'references': List(Value('string'))}, usage: """
Produces labelling scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions: List of List of predicted labels (Estimated targets as returned by a tagger)
    references: List of List of reference labels (Ground truth (correct) target values)
    suffix: True if the IOB prefix is after type, False otherwise. default: False
    scheme: Specify target tagging scheme. Should be one of ["IOB1", "IOB2", "IOE1", "IOE2", "IOBES", "BILOU"].
        default: None
    mode: Whether to count correct entity labels with incorrect I/B tags as true positives or not.
        If you want to only count exact matches, pass mode="strict". default: None.
    sample_weight: Array-like of shape (n_samples,), weights for individual samples. default: None
    zero_division: Which value to substitute as a

In [20]:
import numpy as np

def eval_metric(pred):
    preds, labels = pred
    preds = np.argmax(preds, axis=-1)

    # 这里 zip(prediction, label) 应该用原始的预测和标签
    true_labels = [
        [label_list[l] for l in label if l != -100]
        for label in labels
    ]
    pred_labels = [
        [label_list[p] for p, l in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, labels)
    ]

    result = seqeval.compute(predictions=pred_labels, references=true_labels, scheme='IOB2')

    return {'f1': result['overall_f1']}

## Step 06 Config training parameters

In [21]:
train_args = TrainingArguments(
    output_dir='models_for_ner',
    per_device_train_batch_size=64,
    per_device_eval_batch_size=128,
    # gradient_checkpointing=True,
    # gradient_accumulation_steps=64,
    # optim='adafactor',
    eval_strategy='epoch',
    save_strategy='epoch',
    metric_for_best_model='f1',
    load_best_model_at_end=True,
    logging_steps=50,
    num_train_epochs=3,
)

## Step 07 Create trainer

In [22]:
trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_ds_dict['train'],
    eval_dataset=tokenized_ds_dict['validation'],
    compute_metrics=eval_metric,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
)

## Step 08 Train the model

In [23]:
trainer.train()

/home/ubuntu/MyFiles/GitHub/demos/transformers/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.201500,0.022896,0.913373
2,0.022700,0.018849,0.934665
3,0.017100,0.017801,0.943514


/home/ubuntu/MyFiles/GitHub/demos/transformers/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/ubuntu/MyFiles/GitHub/demos/transformers/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=246, training_loss=0.0582680966311354, metrics={'train_runtime': 170.4274, 'train_samples_per_second': 367.282, 'train_steps_per_second': 1.443, 'total_flos': 4085886105396480.0, 'train_loss': 0.0582680966311354, 'epoch': 3.0})

In [24]:
trainer.evaluate(eval_dataset=tokenized_ds_dict['test'])

/home/ubuntu/MyFiles/GitHub/demos/transformers/.venv/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.024191275238990784,
 'eval_f1': 0.9285666909862704,
 'eval_runtime': 7.0917,
 'eval_samples_per_second': 653.864,
 'eval_steps_per_second': 1.41,
 'epoch': 3.0}

## Step 09 Model predicting

In [25]:
from transformers import pipeline

In [26]:
model.config.id2label = {idx: label for idx, label in enumerate(label_list)}
model.config

BertConfig {
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "torch

In [27]:
ner_pipe = pipeline('token-classification', model=model, tokenizer=tokenizer, device='cuda:0', aggregation_strategy='simple')

Device set to use cuda:0


In [28]:
res = ner_pipe('小明在北京上班')
res

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity_group': 'PER',
  'score': np.float32(0.57519144),
  'word': '明',
  'start': 1,
  'end': 2},
 {'entity_group': 'LOC',
  'score': np.float32(0.9954323),
  'word': '北 京',
  'start': 3,
  'end': 5}]

In [30]:
ner_result = {}
x = '小明在北京上班'
for r in res:
    if r['entity_group'] not in ner_result:
        ner_result[r['entity_group']] = []
    ner_result[r['entity_group']].append(x[r['start']:r['end']])

ner_result

{'PER': ['明'], 'LOC': ['北京']}